In [1]:
import pandas as pd
import numpy as np
import stat
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
#analysis = pd.read_csv("FF_HEALTH_DC3_ANALYSIS.csv")
roster = pd.read_csv("ME292C_ROSTER.csv")
roster = roster.rename({'Team Number': 'TeamNumber'}, axis=1)
dictionary = pd.read_csv("ME292C_QUESTION_DICTIONARY.csv")
raw = pd.read_csv("ME292C_CHECKIN1_RAW_V2.csv")
#numerical_data = pd.read_csv("full_cleaned.csv")[[
full_cleaned_data = pd.read_csv("full_cleaned.csv")
full_cleaned_data = full_cleaned_data.rename({'Team Number': 'TeamNumber'}, axis=1)

In [3]:
full_cleaned_data

,Unnamed: 0,Email,TeamNumber,TeammateNumber,RecipientLastName,RecipientFirstName,RecipientEmail,32,33,33.4.TEXT,...,18.2,18.3,18.4,18.5,19.1,19.2,19.3,19.4,19.5,25
0,0,chunming_chen@berkeley.edu,1.0,1.0,No Response,No Response,chunming_chen@berkeley.edu,2,2,No Response,...,1.0,3.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0,1
1,1,clairewhsu@berkeley.edu,1.0,2.0,No Response,No Response,clairewhsu@berkeley.edu,1,2,No Response,...,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0,1
2,2,ginger.lau@berkeley.edu,1.0,3.0,No Response,No Response,ginger.lau@berkeley.edu,2,2,No Response,...,-0.0,3.0,1.0,-0.0,2.0,2.0,2.0,1.0,2.0,1
3,3,mingyu_zhu@berkeley.edu,1.0,4.0,No Response,No Response,mingyu_zhu@berkeley.edu,1,2,No Response,...,1.0,3.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0,1
4,4,tddo0o@berkeley.edu,1.0,5.0,No Response,No Response,tddo0o@berkeley.edu,3,3,No Response,...,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,1
5,5,quentin.daurat@berkeley.edu,6.0,1.0,No Response,No Response,quentin.daurat@berkeley.edu,1,1,No Response,...,3.0,3.0,3.0,No Response,3.0,3.0,3.0,3.0,No Response,1
6,6,rupesh@berkeley.edu,6.0,2.0,No Response,No Response,rupesh@berkeley.edu,1,4,Weekly In-person meetings + constant virtual c...,...,3.0,3.0,3.0,No Response,3.0,3.0,3.0,3.0,No Response,1
7,7,sean_chu@berkeley.edu,6.0,3.0,No Response,No Response,sean_chu@berkeley.edu,1,1,No Response,...,3.0,3.0,3.0,No Response,3.0,3.0,3.0,3.0,No Response,1
8,8,ysn_2003@berkeley.edu,6.0,4.0,No Response,No Response,ysn_2003@berkeley.edu,2,1,No Response,...,2.0,2.0,2.0,No Response,2.0,2.0,2.0,2.0,No Response,2
9,9,asun23@berkeley.edu,7.0,1.0,No Response,No Response,asun23@berkeley.edu,2,2,No Response,...,3.0,2.0,3.0,No Response,3.0,2.0,3.0,3.0,No Response,1


In [4]:
quant = [str(x) for x in list(dictionary[dictionary["type"] == 'quantitative']["question_id"])]

numerical_data = full_cleaned_data[['TeamNumber', 'TeammateNumber','2.1',
 '2.2',
 '2.3',
 '4.1',
 '4.2',
 '4.3',
 '4.4',
 '5.1',
 '5.2',
 '5.3',
 '5.4',
 '5.5',
 '16.1',
 '16.2',
 '16.3',
 '16.4',
 '16.5',
 '17.1',
 '17.2',
 '17.3',
 '17.4',
 '17.5',
 '18.1',
 '18.2',
 '18.3',
 '18.4',
 '18.5',
 '19.1',
 '19.2',
 '19.3',
 '19.4',
 '19.5']]
numerical_data = numerical_data.replace({"No Response" : np.nan})
numerical_data.head()

,TeamNumber,TeammateNumber,2.1,2.2,2.3,4.1,4.2,4.3,4.4,5.1,...,18.1,18.2,18.3,18.4,18.5,19.1,19.2,19.3,19.4,19.5
0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,20,...,2.0,1.0,3.0,1.0,2.0,3.0,3.0,3.0,3.0,3.0
1,1.0,2.0,1.0,1.0,1.0,1.0,-1.0,2.0,1.0,13,...,3.0,3.0,3.0,2.0,3.0,3.0,3.0,3.0,2.0,3.0
2,1.0,3.0,1.0,1.0,1.0,1.0,-2.0,2.0,1.0,7,...,-0.0,-0.0,3.0,1.0,-0.0,2.0,2.0,2.0,1.0,2.0
3,1.0,4.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,19,...,1.0,1.0,3.0,1.0,1.0,3.0,3.0,3.0,3.0,3.0
4,1.0,5.0,-1.0,-1.0,-2.0,3.0,-1.0,1.0,3.0,16,...,2.0,2.0,3.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0


In [5]:
numerical_data = numerical_data.astype(float)
numerical_data["Email"] = full_cleaned_data["Email"]

In [6]:
roster.head()

,Email,First,Last,Assignment,TeamNumber,FullName,TeammateNumber,Teammate1,Teammate2,Teammate3,Teammate4,Teammate5
0,chunming_chen@berkeley.edu,Chun-Ming,Chen,1 - Personalizing Clothing Sizes,1,Chun-Ming Chen,1,Chun-Ming Chen,Claire Hsu,ginger lau,Mingyu Zhu,Zhengrong Tang
1,clairewhsu@berkeley.edu,Claire,Hsu,1 - Personalizing Clothing Sizes,1,Claire Hsu,2,Chun-Ming Chen,Claire Hsu,ginger lau,Mingyu Zhu,Zhengrong Tang
2,ginger.lau@berkeley.edu,ginger,lau,1 - Personalizing Clothing Sizes,1,ginger lau,3,Chun-Ming Chen,Claire Hsu,ginger lau,Mingyu Zhu,Zhengrong Tang
3,mingyu_zhu@berkeley.edu,Mingyu,Zhu,1 - Personalizing Clothing Sizes,1,Mingyu Zhu,4,Chun-Ming Chen,Claire Hsu,ginger lau,Mingyu Zhu,Zhengrong Tang
4,tddo0o@berkeley.edu,Zhengrong,Tang,1 - Personalizing Clothing Sizes,1,Zhengrong Tang,5,Chun-Ming Chen,Claire Hsu,ginger lau,Mingyu Zhu,Zhengrong Tang


In [7]:
#Functions 

def calculate_team_averages(df):
    # Creates a DataFrame with the averages per team for each question
    return df.groupby("TeamNumber").agg(np.nanmean)[numerical_questions]

def get_team_data(team_number):
    # Returns new dataframe containing only TEAM_NUMBER's data
    return numerical_data[numerical_data['TeamNumber'] == team_number]

def get_team_average(team_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    temp_df = get_team_data(team_number)
    return np.nanmean(temp_df[question])

def get_team_std(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's standard deviation on the specified question and should not include self
    temp_df = get_team_data(team_number)
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanstd(temp_df[question])

def get_teammates_average(team_number, teammate_number, question):
    # Gets TEAM_NUMBER's average on the specified question
    # omits data from current student given by TEAMMATE_NUMBER 
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] != teammate_number]
    
    return np.nanmean(temp_df[question])


def get_student_answer(team_number, teammate_number, question):
    # Returns the current student's response for the specified question
    temp_df = get_team_data(team_number)
    
    temp_df = temp_df[temp_df['TeammateNumber'] == teammate_number]
    
    return float(temp_df[question])
    #return temp_df[question]

def get_class_average(question):
    # Returns the class average for QUESTION
    return np.mean(numerical_data[question])

def get_class_stdv(question):
    # Returns the class std dev for QUESTION
    return np.std(numerical_data[question])

## Part 1: Identification Information

In [8]:
#Columns we need
roster = roster.rename({"Assignment" : "TeamName"}, axis=1)
#roster["Full Name"] = roster["First Name"] + ' ' + roster["Last Name"]

In [9]:
#roster = roster.rename({"FullName": "Full Name"}, axis=1)

In [10]:
analysis_v2 = roster[["FullName", "Email", "TeamNumber", "TeammateNumber", "TeamName"]]

In [11]:
analysis_v2 = analysis_v2.sort_values(["TeamNumber", "TeammateNumber"], ascending=True).reset_index(drop=True)

In [12]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,TeamName
0,Chun-Ming Chen,chunming_chen@berkeley.edu,1,1,1 - Personalizing Clothing Sizes
1,Claire Hsu,clairewhsu@berkeley.edu,1,2,1 - Personalizing Clothing Sizes
2,ginger lau,ginger.lau@berkeley.edu,1,3,1 - Personalizing Clothing Sizes
3,Mingyu Zhu,mingyu_zhu@berkeley.edu,1,4,1 - Personalizing Clothing Sizes
4,Zhengrong Tang,tddo0o@berkeley.edu,1,5,1 - Personalizing Clothing Sizes
5,Quentin Daurat,quentin.daurat@berkeley.edu,6,1,6 - Using Passive Hydroponics in Urban Environ...
6,Rupesh Shrestha,rupesh@berkeley.edu,6,2,6 - Using Passive Hydroponics in Urban Environ...
7,Sean Chu,sean_chu@berkeley.edu,6,3,6 - Using Passive Hydroponics in Urban Environ...
8,Yashvardhan Saravan Nalina,ysn_2003@berkeley.edu,6,4,6 - Using Passive Hydroponics in Urban Environ...
9,Angela Sun,asun23@berkeley.edu,7,1,7 - Preemptively Detect Blood Sugar Changes fo...


## Part 2: Skills Assessment

In [13]:
...

Ellipsis

## Part 3: Survey Questionnairre

In [14]:
question_shorthand = {}
question_shorthand[2.1] = ['SharedGoalTeamAvg', 'SharedGoalClassAvg', 'SharedGoalClassStDev']
question_shorthand[2.2] = ['IndvGoalTeamAvg', 'IndvGoalClassAvg', 'IndvGoalClassStDev']
question_shorthand[2.3] = ['SupportTeamAvg', 'SupportClassAvg', 'SupportClassStDev']
question_shorthand[3.0] = ['Goal1', 'Goal2', 'Goal3', 'Goal4', 'Goal5']
question_shorthand[4.1] = ['CommunicateTeamAvg', 'CommunicateClassAvg', 'CommunicateClassStDev']
question_shorthand[4.2] = ['WorkAllocTeamAvg', 'WorkAllocClassAvg', 'WorkAllocClassStDev']
question_shorthand[4.3] = ['RoleTeamAvg', 'RoleClassAvg', 'RoleClassStDev']
question_shorthand[4.4] = ['EnjoyTeamAvg', 'EnjoyClassAvg', 'EnjoyClassStDev']
question_shorthand[5.0] = ['MyPoints', 'TeammateAvgPoints', 'ExpectedPoints', 'Percent', 'AveragePointsAllocated', 'StDevPointsAllocated']
question_shorthand[5.1] = 'Work Contribution Teammate 1'
question_shorthand[5.2] = 'Work Contribution Teammate 2'
question_shorthand[5.3] = 'Work Contribution Teammate 3'
question_shorthand[5.4] = 'Work Contribution Teammate 4'
question_shorthand[5.5] = 'Work Contribution Teammate 5'
#question_shorthand[5.6] = 'Work Contribution Teammate 6' #might need to increase this based on team size
question_shorthand[6.0] = ['DidWellMe','DidWell1','DidWell2','DidWell3','DidWell4', 'DidWell5' ]
question_shorthand[7] = ['DoDiffMe', 'DoDiff1', 'DoDiff2', 'DoDiff3', 'DoDiff4', 'DoDiff5']
#question_shorthand[9.1] = 'ImproveAmt1'
#question_shorthand[9.2] = 'ImproveAmt2'
#question_shorthand[9.3] = 'ImproveAmt3'
#question_shorthand[10.1] = ["AskFeedbackClassAvg", "AskFeedbackClassStDev"]
#question_shorthand[10.2] = ["AskHelpClassAvg", "AskHelpClassStDev"]
#question_shorthand[10.3] = ["ValueContrClassAvg", "ValueContrClassStdev"]
#question_shorthand[10.4] = ["ReflectClassAvg", "ReflectClassStDev"]
#questions 11-13 are just imported as questions themselves as columns
question_shorthand[14] = ["ContributeMe", "Contribute1", "Contribute2", "Contribute3", "Contribute4", "Contribute5"]
question_shorthand[15] = ['ImproveMe', 'Improve1', 'Improve2', 'Improve3', 'Improve4', 'Improve5']
question_shorthand[16] = ['InitiativeMe', 'InitiativeTeammateAvg', 'InitiativeClassAvg', 'InitiativeClassStDev']
question_shorthand[17] = ['CommunicateMe', 'CommunicateTeammateAvg', 'Communicate2ClassAvg', 'Communicate2ClassStDev']
question_shorthand[18] = ['ExpertiseMe', 'ExpertiseTeammateAvg', 'ExpertiseClassAvg', 'ExpertiseClassStDev']
question_shorthand[19] = ['RespectMe', 'RespectTeammateAvg', 'RespectClassAvg', 'RespectClassStDev']
#question_shorthand[38.1] = 'ImproveAmt1'
#question_shorthand[38.2] = 'ImproveAmt2'
#question_shorthand[38.3] = 'ImproveAmt3'

### 2.1

In [15]:
#Question 2.1

question_2_1 = dictionary[dictionary["question_id"] == 2.1].reset_index()["question"][0]
analysis_v2[question_2_1] = numerical_data["2.1"]

In [16]:
#Question 2.1 team average
question_2_1_0 = question_shorthand[2.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.1"), 1))

analysis_v2[question_2_1_0] = lst_responses

In [17]:
#Question 2.1 class avg
question_2_1_1 = question_shorthand[2.1][1]

analysis_v2[question_2_1_1] = round(get_class_average("2.1"), 1)

In [18]:


#Question 2.1 class std
question_2_1_2 = question_shorthand[2.1][2]

analysis_v2[question_2_1_2] = round(get_class_stdv("2.1"), 1)

### 2.2

In [19]:
#Question 2.2

question_2_2 = dictionary[dictionary["question_id"] == 2.2].reset_index()["question"][0]
analysis_v2[question_2_2] = numerical_data["2.2"]

In [20]:
#Question 2.2 team average
question_2_2_0 = question_shorthand[2.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.2"), 1))

analysis_v2[question_2_2_0] = lst_responses

In [21]:
#Question 2.2 class avg
question_2_2_1 = question_shorthand[2.2][1]

analysis_v2[question_2_2_1] = round(get_class_average("2.2"), 1)

In [22]:
#Question 2.2 class std
question_2_2_2 = question_shorthand[2.2][2]

analysis_v2[question_2_2_2] = round(get_class_stdv("2.2"), 1)

In [23]:
#analysis_v2[analysis_v2["TeamNumber"] == 1022]

### 2.3

In [24]:
#Question 2.3

question_2_3 = dictionary[dictionary["question_id"] == 2.3].reset_index()["question"][0]
analysis_v2[question_2_3] = numerical_data["2.3"]

In [25]:
#Question 2.3 team average
question_2_3_0 = question_shorthand[2.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "2.3"), 1))

analysis_v2[question_2_3_0] = lst_responses

In [26]:
#Question 2.3 class avg
question_2_3_1 = question_shorthand[2.3][1]

analysis_v2[question_2_3_1] = round(get_class_average("2.3"), 1)

In [27]:
#Question 2.3 class std
question_2_3_2 = question_shorthand[2.3][2]

analysis_v2[question_2_3_2] = round(get_class_stdv("2.3"), 1)

### 3

In [28]:
question_3 = dictionary[dictionary["question_id"] == 3.0].reset_index()["question"][0]

In [29]:
analysis_v2[question_3] = full_cleaned_data["3"]

In [30]:
#goal 1
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 1].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_1 = question_shorthand[3.0][0]
analysis_v2[question_3_1] = lst_responses

In [31]:
#goal 2
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 2].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_2 = question_shorthand[3.0][1]
analysis_v2[question_3_2] = lst_responses

In [32]:
#goal 3
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 3].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_3 = question_shorthand[3.0][2]
analysis_v2[question_3_3] = lst_responses

In [33]:
#goal 4
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 4].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_4 = question_shorthand[3.0][3]
analysis_v2[question_3_4] = lst_responses

In [34]:
#goal 5
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 5].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_5 = question_shorthand[3.0][4]
analysis_v2[question_3_5] = lst_responses

In [35]:
#goal 6
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    df_question_at_hand = team_df[team_df["TeammateNumber"] == 6].reset_index()["3"]
    if len(df_question_at_hand) == 0:
        lst_responses.append(np.nan)
    else:
        lst_responses.append(df_question_at_hand[0])
    
question_3_6 = question_shorthand[3.0][5]
analysis_v2[question_3_6] = lst_responses

IndexError: list index out of range

### 4.1

In [36]:
#Question 4.1

question_4_1 = dictionary[dictionary["question_id"] == 4.1].reset_index()["question"][0]
analysis_v2[question_4_1] = numerical_data["4.1"]

In [37]:
#analysis_v2

In [38]:
#Question 4.1 team average
question_4_1_0 = question_shorthand[4.1][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.1"), 1))

analysis_v2[question_4_1_0] = lst_responses

In [39]:
#analysis_v2

In [40]:
#Question 4.1 class avg
question_4_1_1 = question_shorthand[4.1][1]

analysis_v2[question_4_1_1] = round(get_class_average("4.1"), 1)

In [41]:
#Question 4.1 class std
question_4_1_2 = question_shorthand[4.1][2]

analysis_v2[question_4_1_2] = round(get_class_stdv("4.1"), 1)

### 4.2

In [42]:
#Question 4.2

question_4_2 = dictionary[dictionary["question_id"] == 4.2].reset_index()["question"][0]
analysis_v2[question_4_2] = numerical_data["4.2"]

In [43]:
#Question 4.2 team average
question_4_2_0 = question_shorthand[4.2][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.2"), 1))

analysis_v2[question_4_2_0] = lst_responses

In [44]:
#Question 4.2 class avg
question_4_2_1 = question_shorthand[4.2][1]

analysis_v2[question_4_2_1] = round(get_class_average("4.2"), 1)

In [45]:
#Question 4.2 class std
question_4_2_2 = question_shorthand[4.2][2]

analysis_v2[question_4_2_2] = round(get_class_stdv("4.2"), 1)

### 4.3

In [46]:
#Question 4.3

question_4_3 = dictionary[dictionary["question_id"] == 4.3].reset_index()["question"][0]
analysis_v2[question_4_3] = numerical_data["4.3"]

In [47]:
#Question 4.1 team average
question_4_3_0 = question_shorthand[4.3][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.3"), 1))

analysis_v2[question_4_3_0] = lst_responses

In [48]:
#Question 4.3 class avg
question_4_3_1 = question_shorthand[4.3][1]

analysis_v2[question_4_3_1] = round(get_class_average("4.3"), 1)

In [49]:
#Question 4.3 class std
question_4_3_2 = question_shorthand[4.3][2]

analysis_v2[question_4_3_2] = round(get_class_stdv("4.3"), 1)

### 4.4

In [50]:
#Question 4.4

question_4_4 = dictionary[dictionary["question_id"] == 4.4].reset_index()["question"][0]
analysis_v2[question_4_4] = numerical_data["4.4"]

In [51]:
#Question 4.4 team average
question_4_4_0 = question_shorthand[4.4][0]
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    lst_responses.append(round(get_team_average(student_team_number, "4.4"), 1))

analysis_v2[question_4_4_0] = lst_responses

In [52]:
#Question 4.4 class avg
question_4_4_1 = question_shorthand[4.4][1]

analysis_v2[question_4_4_1] = round(get_class_average("4.4"), 1)

In [53]:
#Question 4.4 class std
question_4_4_2 = question_shorthand[4.4][2]

analysis_v2[question_4_4_2] = round(get_class_stdv("4.4"), 1)

### 5

In [54]:
#Question 5.1-5.6. will need to change this according to how many people are on a team
question_5_1 = dictionary[dictionary["question_id"] == 5.1].reset_index()["question"][0]
question_5_2 = dictionary[dictionary["question_id"] == 5.2].reset_index()["question"][0]
question_5_3 = dictionary[dictionary["question_id"] == 5.3].reset_index()["question"][0]
question_5_4 = dictionary[dictionary["question_id"] == 5.4].reset_index()["question"][0]
question_5_5 = dictionary[dictionary["question_id"] == 5.5].reset_index()["question"][0]
#question_5_6 = dictionary[dictionary["question_id"] == 5.6].reset_index()["question"][0]

analysis_v2[question_5_1] = full_cleaned_data[["5.1"]]
analysis_v2[question_5_2] = full_cleaned_data[["5.2"]]
analysis_v2[question_5_3] = full_cleaned_data[["5.3"]]
analysis_v2[question_5_4] = full_cleaned_data[["5.4"]]
analysis_v2[question_5_5] = full_cleaned_data[["5.5"]]
#analysis_v2[question_5_6] = full_cleaned_data[["5.6"]]

In [55]:
#Calculations

#MyPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    student_df = team_df[team_df["TeammateNumber"] == student_teammate_number].reset_index()
    question_needed = '5.' + str(int(student_teammate_number))
    #print(student_df)
    lst_responses.append(student_df[question_needed].reset_index()[question_needed][0])
    
question_5_a = question_shorthand[5][0]
analysis_v2[question_5_a] = lst_responses


In [56]:
#TeammateAvgPoints
lst_responses = []
for student_email in analysis_v2["Email"]:
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == team]["TeammateNumber"])
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    question_needed = '5.' + str(int(student_teammate_number))
    lst_responses.append(round(get_teammates_average(student_team_number, student_teammate_number, question_needed), 1))

question_5_b = question_shorthand[5][1]
analysis_v2[question_5_b] = lst_responses



In [57]:
#ExpectedPoints
lst_responses = []
for student_team_number in analysis_v2["TeamNumber"]:
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    lst_responses.append(round(100/team_size, 1))

question_5_c = question_shorthand[5][2]
analysis_v2[question_5_c] = lst_responses

In [58]:
#Percent
question_5_d = question_shorthand[5][3]
analysis_v2[question_5_d] = round((analysis_v2[question_5_b] / analysis_v2[question_5_c]) * 100, 1)

In [59]:
#AveragePointsAllocated
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4", "5.5"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    sum_points = sum(temp_q5_df.reset_index().drop("index", axis=1).iloc[int(student_teammate_number) - 1])
    lst_responses.append(round(sum_points/team_size, 1))

question_5_e = question_shorthand[5][4]
analysis_v2[question_5_e] = lst_responses

In [60]:
#StDevPointsAllocated
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    
    temp_q5_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number][["5.1", "5.2", "5.3", "5.4", "5.5"]]
    temp_q5_df = temp_q5_df.replace({"No Response" : np.nan}).astype(float)
    arr = temp_q5_df.reset_index().drop("index", axis=1).dropna().to_numpy().flatten()
    std = np.std(arr[arr != 0])
    lst_responses.append(round(std, 1))

question_5_f = question_shorthand[5][5]
analysis_v2[question_5_f] = lst_responses

### 6

In [61]:
#Didwellme 

lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q6_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    response = temp_q6_df[temp_q6_df["TeammateNumber"] == student_teammate_number].reset_index()["6"][0]
    lst_responses.append(response)

question_6_a = question_shorthand[6][0]
analysis_v2[question_6_a] = lst_responses

In [62]:
#DidWell1 thru DidWell6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DidWellMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
lst_responses_5 = []
lst_responses_6 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DidWellMe"]
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(team_series[5])

question_6_b = question_shorthand[6][1]
question_6_c = question_shorthand[6][2]
question_6_d = question_shorthand[6][3]
question_6_e = question_shorthand[6][4]
question_6_f = question_shorthand[6][5]
analysis_v2[question_6_b] = lst_responses_1
analysis_v2[question_6_c] = lst_responses_2
analysis_v2[question_6_d] = lst_responses_3
analysis_v2[question_6_e] = lst_responses_4
analysis_v2[question_6_f] = lst_responses_5
analysis_v2["DidWell6"] = lst_responses_6

### 7

In [63]:
#DoDiffMe
lst_responses = []
for student_email in analysis_v2["Email"]:
    student_team_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeamNumber"][0]
    #team_size = max(analysis_v2[analysis_v2["TeamNumber"] == student_team_number]["TeammateNumber"])
    student_teammate_number = analysis_v2[analysis_v2["Email"] == student_email].reset_index()["TeammateNumber"][0]
    temp_q7_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == student_team_number]
    #temp_q7_df = temp_q7_df.replace({"No Response" : np.nan}).astype(float)
    response = temp_q7_df[temp_q7_df["TeammateNumber"] == student_teammate_number].reset_index()["7"][0]
    lst_responses.append(response)

question_7_a = question_shorthand[7][0]
analysis_v2[question_7_a] = lst_responses

In [64]:
#DoDiff1 thru DoDiff6

subset_df = analysis_v2[["TeamNumber", "TeammateNumber", "DoDiffMe"]]
#subset_df[subset_df["TeammateNumber"] == 1]
#subset_df

lst_responses_1 = []
lst_responses_2 = []
lst_responses_3 = []
lst_responses_4 = []
lst_responses_5 = []
lst_responses_6 = []

for team in subset_df["TeamNumber"]:#.unique():
    team_series = subset_df[subset_df["TeamNumber"] == team].reset_index()["DoDiffMe"]
    if len(team_series) == 4:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(np.nan)
        lst_responses_6.append(np.nan)
    if len(team_series) == 5:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(np.nan)
    if len(team_series) == 6:
        lst_responses_1.append(team_series[0])
        lst_responses_2.append(team_series[1])
        lst_responses_3.append(team_series[2])
        lst_responses_4.append(team_series[3])
        lst_responses_5.append(team_series[4])
        lst_responses_6.append(team_series[5])

question_7_b = question_shorthand[7][1]
question_7_c = question_shorthand[7][2]
question_7_d = question_shorthand[7][3]
question_7_e = question_shorthand[7][4]
question_7_f = question_shorthand[7][5]
analysis_v2[question_7_b] = lst_responses_1
analysis_v2[question_7_c] = lst_responses_2
analysis_v2[question_7_d] = lst_responses_3
analysis_v2[question_7_e] = lst_responses_4
analysis_v2[question_7_f] = lst_responses_5
analysis_v2["DoDiff6"] = lst_responses_6

### 9

In [65]:
#analysis_v2[question_shorthand[9.1]] = full_cleaned_data["9.1"]
#analysis_v2[question_shorthand[9.2]] = full_cleaned_data["9.2"]
#analysis_v2[question_shorthand[9.3]] = full_cleaned_data["9.3"]

### 10 (not used here but can use functions below)

In [66]:
#analysis_v2[question_shorthand[10.1][0]] = get_class_average("10.1")
#analysis_v2[question_shorthand[10.1][1]] = get_class_stdv("10.1")

#analysis_v2[question_shorthand[10.2][0]] = get_class_average("10.2")
#analysis_v2[question_shorthand[10.2][1]] = get_class_stdv("10.2")

#analysis_v2[question_shorthand[10.3][0]] = get_class_average("10.3")
#analysis_v2[question_shorthand[10.3][1]] = get_class_stdv("10.3")


### 11

In [67]:
#question_11 = dictionary[dictionary["question_id"] == 11].reset_index()["question"][0]
#analysis_v2[question_11] = full_cleaned_data["11"]

### 12

In [68]:
#question_12 = dictionary[dictionary["question_id"] == 12].reset_index()["question"][0]
#analysis_v2[question_12] = full_cleaned_data["12"]

### 14

In [69]:
#14.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '14.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_14_me = question_shorthand[14][0]
analysis_v2[question_14_me] = lst_responses

In [70]:
#14

question_14_1 = question_shorthand[14][1]
question_14_2 = question_shorthand[14][2]
question_14_3 = question_shorthand[14][3]
question_14_4 = question_shorthand[14][4]
question_14_5 = question_shorthand[14][5]
#question_14_6 = question_shorthand[14][6]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["14." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 6 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
    #feedback_6.append(all_feedback[5])
    
analysis_v2[question_14_1] = feedback_1
analysis_v2[question_14_2] = feedback_2
analysis_v2[question_14_3] = feedback_3
analysis_v2[question_14_4] = feedback_4
analysis_v2[question_14_5] = feedback_5
#analysis_v2[question_14_6] = feedback_6

### 15

In [71]:
#15.1 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '15.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_15_me = question_shorthand[15][0]
analysis_v2[question_15_me] = lst_responses

In [72]:
#15

question_15_1 = question_shorthand[15][1]
question_15_2 = question_shorthand[15][2]
question_15_3 = question_shorthand[15][3]
question_15_4 = question_shorthand[15][4]
question_15_5 = question_shorthand[15][5]
#question_15_6 = question_shorthand[15][6]

feedback_1 = []
feedback_2 = []
feedback_3 = []
feedback_4 = []
feedback_5 = [] 
#feedback_6 = []#need to adjust to max team size

#possibilities = ["14.1", "14.2", "14.3", "14.4", "14.5", "14.6"]
for i in range(len(full_cleaned_data)):
    my_teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    my_team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    my_team_df = full_cleaned_data[full_cleaned_data["TeamNumber"] == my_team_number]
    list_feedback_for_me = list(my_team_df["15." + str(my_teammate_number)])
    num_teammates = len(list_feedback_for_me)
    extra_needed = 6 - num_teammates
    placeholders = [''] * extra_needed
    all_feedback = list_feedback_for_me + placeholders

    feedback_1.append(all_feedback[0])
    feedback_2.append(all_feedback[1])
    feedback_3.append(all_feedback[2])
    feedback_4.append(all_feedback[3])
    feedback_5.append(all_feedback[4])
    #feedback_6.append(all_feedback[5])
    
analysis_v2[question_15_1] = feedback_1
analysis_v2[question_15_2] = feedback_2
analysis_v2[question_15_3] = feedback_3
analysis_v2[question_15_4] = feedback_4
analysis_v2[question_15_5] = feedback_5
#analysis_v2[question_15_6] = feedback_6

### 16

In [73]:
#16 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_16_me = question_shorthand[16][0]
analysis_v2[question_16_me] = lst_responses

In [74]:
#Question 16 team average
question_16_tm_avg = question_shorthand[16][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '16.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_16_tm_avg] = lst_responses

In [75]:
#Question 16 class avg
question_16_class_avg = question_shorthand[16][2]
class_avg = np.nanmean(full_cleaned_data[["16.1", "16.2", "16.3", "16.4", "16.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_16_class_avg] = round(class_avg, 1)

In [76]:
#Question 16 class std
question_16_class_std = question_shorthand[16][3]
class_std_arr = full_cleaned_data[["16.1", "16.2", "16.3", "16.4", "16.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_16_class_std] = round(np.std(class_std_arr_no_na), 1)

### 17

In [77]:
#17 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_17_me = question_shorthand[17][0]
analysis_v2[question_17_me] = lst_responses

In [78]:
#Question 17 team average
question_17_tm_avg = question_shorthand[17][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '17.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_17_tm_avg] = lst_responses

In [79]:
#analysis_v2

In [80]:
#Question 17 class avg
question_17_class_avg = question_shorthand[17][2]
class_avg = np.nanmean(full_cleaned_data[["17.1", "17.2", "17.3", "17.4", "17.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_17_class_avg] = round(class_avg, 1)

In [81]:
#Question 17 class std
question_17_class_std = question_shorthand[17][3]
class_std_arr = full_cleaned_data[["17.1", "17.2", "17.3", "17.4", "17.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_17_class_std] = round(np.std(class_std_arr_no_na), 1)

### 18

In [82]:
#18 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_18_me = question_shorthand[18][0]
analysis_v2[question_18_me] = lst_responses

In [83]:
#Question 18 team average
question_18_tm_avg = question_shorthand[18][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '18.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_18_tm_avg] = lst_responses

In [84]:
#Question 18 class avg
question_18_class_avg = question_shorthand[18][2]
class_avg = np.nanmean(full_cleaned_data[["18.1", "18.2", "18.3", "18.4", "18.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_18_class_avg] = round(class_avg, 1)

In [85]:
#Question 18 class std
question_18_class_std = question_shorthand[18][3]
class_std_arr = full_cleaned_data[["18.1", "18.2", "18.3", "18.4", "18.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_18_class_std] = round(np.std(class_std_arr_no_na), 1)

### 19

In [86]:
#19 (me)

lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(full_cleaned_data.iloc[i][question_needed])

question_19_me = question_shorthand[19][0]
analysis_v2[question_19_me] = lst_responses

In [87]:
#Question 19 team average
question_19_tm_avg = question_shorthand[19][1]
lst_responses = []

for i in range(len(full_cleaned_data)):
    teammate_number = int(full_cleaned_data.iloc[i]["TeammateNumber"])
    team_number = int(full_cleaned_data.iloc[i]["TeamNumber"])
    question_needed = '19.' + str(teammate_number)
    lst_responses.append(round(get_teammates_average(team_number, teammate_number, question_needed), 1))

analysis_v2[question_19_tm_avg] = lst_responses

In [88]:
#Question 19 class avg
question_19_class_avg = question_shorthand[19][2]
class_avg = np.nanmean(full_cleaned_data[["19.1", "19.2", "19.3", "19.4", "19.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten())
analysis_v2[question_19_class_avg] = round(class_avg, 1)

In [89]:
#Question 19 class std
question_19_class_std = question_shorthand[19][3]
class_std_arr = full_cleaned_data[["19.1", "19.2", "19.3", "19.4", "19.5"]].replace({"No Response" : np.nan}).astype(float).to_numpy().flatten()
class_std_arr_no_na = class_std_arr[~np.isnan(class_std_arr)]
analysis_v2[question_19_class_std] = round(np.std(class_std_arr_no_na), 1)

## Add extra questions

In [90]:
analysis_v2["Q33"] = full_cleaned_data['33']
analysis_v2["Q33_TEXT"] = full_cleaned_data['33.4.TEXT']
analysis_v2["Q25"] = full_cleaned_data['25']

In [91]:
analysis_v2[question_shorthand[38.1]] = full_cleaned_data["38.1"]
analysis_v2[question_shorthand[38.2]] = full_cleaned_data["38.2"]
analysis_v2[question_shorthand[38.3]] = full_cleaned_data["38.3"]

KeyError: '38.1'

In [92]:
analysis_v2

,FullName,Email,TeamNumber,TeammateNumber,TeamName,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,...,ExpertiseTeammateAvg,ExpertiseClassAvg,ExpertiseClassStDev,RespectMe,RespectTeammateAvg,RespectClassAvg,RespectClassStDev,Q33,Q33_TEXT,Q25
0,Chun-Ming Chen,chunming_chen@berkeley.edu,1,1,1 - Personalizing Clothing Sizes,2.0,1.0,2.2,0.9,2.0,...,1.5,2.4,0.9,3.0,2.8,2.6,0.7,2,No Response,1
1,Claire Hsu,clairewhsu@berkeley.edu,1,2,1 - Personalizing Clothing Sizes,1.0,1.0,2.2,0.9,1.0,...,1.0,2.4,0.9,3.0,2.8,2.6,0.7,2,No Response,1
2,ginger lau,ginger.lau@berkeley.edu,1,3,1 - Personalizing Clothing Sizes,1.0,1.0,2.2,0.9,1.0,...,3.0,2.4,0.9,2.0,3.0,2.6,0.7,2,No Response,1
3,Mingyu Zhu,mingyu_zhu@berkeley.edu,1,4,1 - Personalizing Clothing Sizes,2.0,1.0,2.2,0.9,2.0,...,1.5,2.4,0.9,3.0,2.2,2.6,0.7,2,No Response,1
4,Zhengrong Tang,tddo0o@berkeley.edu,1,5,1 - Personalizing Clothing Sizes,-1.0,1.0,2.2,0.9,-1.0,...,1.5,2.4,0.9,3.0,2.8,2.6,0.7,3,No Response,1
5,Quentin Daurat,quentin.daurat@berkeley.edu,6,1,6 - Using Passive Hydroponics in Urban Environ...,3.0,2.8,2.2,0.9,3.0,...,2.7,2.4,0.9,3.0,2.7,2.6,0.7,1,No Response,1
6,Rupesh Shrestha,rupesh@berkeley.edu,6,2,6 - Using Passive Hydroponics in Urban Environ...,3.0,2.8,2.2,0.9,3.0,...,2.7,2.4,0.9,3.0,2.7,2.6,0.7,4,Weekly In-person meetings + constant virtual c...,1
7,Sean Chu,sean_chu@berkeley.edu,6,3,6 - Using Passive Hydroponics in Urban Environ...,2.0,2.8,2.2,0.9,2.0,...,2.7,2.4,0.9,3.0,2.7,2.6,0.7,1,No Response,1
8,Yashvardhan Saravan Nalina,ysn_2003@berkeley.edu,6,4,6 - Using Passive Hydroponics in Urban Environ...,3.0,2.8,2.2,0.9,3.0,...,3.0,2.4,0.9,2.0,3.0,2.6,0.7,1,No Response,2
9,Angela Sun,asun23@berkeley.edu,7,1,7 - Preemptively Detect Blood Sugar Changes fo...,2.0,2.3,2.2,0.9,3.0,...,3.0,2.4,0.9,3.0,3.0,2.6,0.7,2,No Response,1


In [93]:
analysis_v2.to_csv("ME292C_CHECKIN1_MACROANALYSIS_v2.csv")